In [1]:
library(h2o)
h2o.init(nthreads = -1, max_mem_size = '90G')


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following object is masked from ‘package:RevoScaleR’:

    colnames

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc




H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpEn2deT/h2o_capsops_started_from_r.out
    /tmp/RtmpEn2deT/h2o_capsops_started_from_r.err


Starting H2O JVM and connecting: ....... Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         6 seconds 202 milliseconds 
    H2O cluster version:        3.16.0.2 
    H2O cluster version age:    3 months and 5 days  
    H2O cluster name:           H2O_started_from_R_capsops_mco465 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   80.00 GB 
    H2O cluster total cores:    16 
    H2O cluster allowed cores:  16 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.4.1 (20

In [30]:
df <- h2o.importFile(path = normalizePath("~/mandy/imputed_17_columns.csv")
                           , destination_frame = "predictors")

df$C1 <- NULL

  |======================================================================| 100%


In [31]:
split <- h2o.splitFrame(df, ratios = c(0.5, 0.2), seed = -1)
training = split[[1]]
validation = split[[2]]

In [32]:
names(training)

[1] "score_C4"                         "EstMarketValue_C1"               
 [3] "finscr_C4"                        "CEN_tr_pctOwnOccSecondMort"      
 [5] "CEN_bg_pctSeasonalHousingUnits"   "CEN_bg_pctHHincomeLT15K"         
 [7] "CEN_tr_pctHHInvestIncome"         "CEN_tr_pctHHSocialSecurityIncome"
 [9] "CEN_bg_pctLiveAloneHH"            "CEN_bg_pctConstructionIndustry"  
[11] "CEN_tr_pctHSGrad"                 "iat89_C4"                        
[13] "imt01_C4"                         "IssAgeALB"                       
[15] "HealthScore_C5"                   "Length.of.Residence_num"         
[17] "Target.Narrow.Band.Income_num"

In [33]:
x = names(training)[!(names(training) %in% c("CEN_bg_pctConstructionIndustry"))]

In [34]:
fit_kmeans <- function(training_frame, k, x, init){
    kmeans_fit <- h2o.kmeans(training_frame = training_frame, x, validation_frame = validation, nfolds = 8,
                             standardize = TRUE, seed = 1234, k = k, init = init)
    return(kmeans_fit)
}

In [35]:
k = 7
model <- fit_kmeans(training, k, x, "PlusPlus")

  |======================================================================| 100%


In [36]:
fit = h2o.predict(object = model, newdata = df)

  |======================================================================| 100%


In [37]:
df$kmeans_7 = fit

In [38]:
df_rframe = as.data.frame(df)

In [39]:
table(df_rframe$kmeans_7)


      0       1       2       3       4       5       6 
 232467  433723 1252364  271723  941149  595529 1005743 

In [40]:
df_score_c4 <- boxplot(score_C4~kmeans_7, df_rframe, plot = FALSE)$stats
df_EstMarketValue_C1 <- boxplot(EstMarketValue_C1~kmeans_7, df_rframe, plot = FALSE)$stats
df_CEN_bg_pctHHincomeLT15K <- boxplot(CEN_bg_pctHHincomeLT15K~kmeans_7, df_rframe, plot = FALSE)$stats
df_CEN_tr_pctHSGrad <- boxplot(CEN_tr_pctHSGrad~kmeans_7, df_rframe, plot = FALSE)$stats
df_HealthScore_C5 <- boxplot(HealthScore_C5~kmeans_7, df_rframe, plot = FALSE)$stats

In [41]:
save(df_HealthScore_C5, df_CEN_tr_pctHSGrad, df_CEN_bg_pctHHincomeLT15K, df_EstMarketValue_C1, df_score_c4, file = "kmeans_7.RData")